In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,googlemaps,json,ast,boto3,sqlalchemy,pytz
from datetime import datetime

from termcolor import colored
from ast import literal_eval
import seaborn as sns
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from carvi import jay

In [4]:
trip_date = '2018-11-23'

qc_list = jay.qc_trip_list(trip_date)
qc_list.head(10)

,camera_id,trip_start,total_distance,location
0,861107036063171,2018-11-23 05:07:35.120,92.042,"41.5121,-90.5732"
1,861107036111616,2018-11-23 05:12:17.097,53.684,"41.5081,-90.5186"
2,861107036108653,2018-11-23 09:02:11.838,156.885,"41.5101,-90.5123"
3,861107036059112,2018-11-24 07:08:24.292,137.604,"41.5548,-90.4097"
4,861107036111616,2018-11-24 07:41:01.510,140.828,"41.5083,-90.5179"
5,861107036077593,2018-11-24 08:34:35.807,295.415,"41.5546,-90.4100"
6,861107036037902,2018-11-24 08:44:55.720,25.373,"41.5122,-90.5757"
7,861107036063171,2018-11-24 09:21:32.703,27.049,"41.5112,-90.5724"
8,861107036037902,2018-11-24 09:55:08.724,5.716,"41.4641,-90.5105"
9,861107036108653,2018-11-24 12:58:18.710,27.953,"41.5087,-90.5163"


In [9]:
df_redshift = jay.redshift_data(imei, trip_start)
jay.trip_info(df_redshift)

Firmware Version CV150S-C1_1.2.0

Trip Start with CarVi Activated
Local Trip Start : America/Chicago 2018-11-24 08:34:35.807
awsS3 Trip Start : UTC		 2018-11-24 14:34:35.807000

Trip End with normal
Local Trip End : America/Chicago 2018-11-24 19:02:09.082
awsS3 Trip End : UTC		 2018-11-25 01:02:09.082000

Data Shape : (37583, 44)
Trip Distance: 295.415 km
Trip Duration: 10 Hours 27 Minutes 33 Seconds


Figure(layout=FigureLayout(height='420px'))

In [10]:
def sample_geo(df):
    
    '''
    This function takes a parameter of a one-trip-dataframe
        : in this script, one-trip data is assigned to 'df_redshift'
    This function prints the information 
        that explains a polygon area and how long the device was staying in the area 
    This function outputs a dictionary that consists of {'polygon area':['latitude', 'longitude']}
    This function implements as follow:
        >> sample_geo(df_redshift)
    '''
    
    garage_zone = [(41.509881, -90.540517),(41.507322, -90.540270),(41.507338, -90.536944),(41.509897, -90.537213)]
    centre_station = [(41.507538, -90.522281),(41.505718, -90.521144),(41.507309, -90.517099),(41.508602, -90.518194)]
    seventh_st = [(41.470531, -90.528824),(41.470499, -90.527773),(41.498171, -90.527494),(41.498284, -90.528116)]
    gas_station = [(41.492974, -90.536377),(41.493472, -90.536361),(41.493458, -90.535827),(41.493026, -90.535848)]

    garage = Polygon(garage_zone)
    centre = Polygon(centre_station)
    sevent = Polygon(seventh_st)
    statio = Polygon(gas_station)

    gar_cnt = 0
    cen_cnt = 0
    sev_cnt = 0
    gas_cnt = 0
    
    gar_lst = []
    cen_lst = []
    sev_lst = []
    gas_lst = []
    
    for loc in df.location:
        if loc != '0.0000,0.0000':
            x = float(loc.split(',')[0])
            y = float(loc.split(',')[1])
            point = Point(x, y)
        
            if garage.contains(point) == True:
                gar_cnt += 1
                gar_lst.append(loc)
            elif centre.contains(point) == True:
                cen_cnt += 1
                cen_lst.append(loc)
            elif sevent.contains(point) == True:
                sev_cnt += 1
                sev_lst.append(loc)
            elif statio.contains(point) == True:
                gas_cnt += 1
                gas_lst.append(loc)

    geo_key = {}
    geo_key['garage'] = gar_lst
    geo_key['centre'] = cen_lst
    geo_key['sevent'] = sev_lst
    geo_key['statio'] = gas_lst
    
    print(colored("Metro Garage","blue",attrs = ['bold']), colored(gar_cnt,"red"), "seconds",
          colored("\nCentre Station", "blue",attrs = ['bold']), colored(cen_cnt,"red"), "seconds",
          colored("\nSeventh St","blue",attrs = ['bold']),colored(sev_cnt,"red"), "seconds",
          colored("\nGas Station","blue",attrs = ['bold']),colored(gas_cnt,"red"), "seconds")
    
    return geo_key
          

In [11]:
geo_key = sample_geo(df_redshift)

Metro Garage 615 seconds 
Centre Station 734 seconds 
Seventh St 0 seconds 
Gas Station 0 seconds


In [12]:
def map_geofence(df, zone):
    
    '''
    This function takes parameters of a data-frame and a polygona area
    This function outputs a google map 
        that shows a geo-fenced polygon area and a plot of latitude and longitude
    This function implements as follow:
        >> map_geofence(geo_key['garage'], garage_zone)
    '''
    
    location_lst = [tuple(ast.literal_eval(loc)) for loc in df]
    cnt_lst = []

    for i in location_lst:
        cnt_lst.append(1)

    new_loc = pd.DataFrame({'location':location_lst,'cnt':cnt_lst})
    #########################################
    polygon = Polygon(zone)
    geo_lst = []
    for i in df:
        x = float(i.split(',')[0])
        y = float(i.split(',')[1])

        point = Point(x, y)
        if polygon.contains(point) == True:
            geo_lst.append(i)
    
    print("total trip", len(df), "seconds")
    print("Trip in GeoFencing area",len(geo_lst), "seconds")
    
    mean = int(round(len(zone)/2,0))
    center = zone[mean]
    
    fig = gmaps.figure(center=center, zoom_level=16)
    polygon = gmaps.Polygon(zone, stroke_color = 'blue', fill_color='blue')
    drawing = gmaps.drawing_layer(features = [gmaps.Polygon(
        zone, stroke_color='blue', fill_color=(255, 0, 132))])
    fig.add_layer(drawing)
    fig.add_layer(gmaps.heatmap_layer(new_loc.location, weights = new_loc['cnt'],
                                      max_intensity = 1, point_radius = 5, opacity = 0.7))
    
    return fig

In [13]:
geo_key.keys()

dict_keys(['garage', 'centre', 'sevent', 'statio'])

## Set the Polygon Area where you want to geo-fence for

#### To set the Polygon Area, you need at least three geographcial coordinates
    
    i.e., three geographical coordinates forms a triangle
          four geographical coordinates forms a quadrangle, etc.

#### In this script, the polygon areas are created with four-coorditates


In [14]:
garage_zone = [(41.509881, -90.540517),(41.507322, -90.540270),(41.507338, -90.536944),(41.509897, -90.537213)]
centre_station = [(41.507538, -90.522281),(41.505718, -90.521144),(41.507309, -90.517099),(41.508602, -90.518194)]
seventh_st = [(41.470531, -90.528824),(41.470499, -90.527773),(41.498171, -90.527494),(41.498284, -90.528116)]
gas_station = [(41.492974, -90.536377),(41.493472, -90.536361),(41.493458, -90.535827),(41.493026, -90.535848)]


# Geo Fencing Example 1

### : *A Geo Fenced Quad City's Garage Area*

In [15]:
map_geofence(geo_key['garage'], garage_zone)

total trip 615 seconds
Trip in GeoFencing area 615 seconds


Figure(layout=FigureLayout(height='420px'))

# Geo Fencing Example 2

### : *A Geo Fenced Bus Transit Station*

In [16]:
map_geofence(geo_key['centre'], centre_station)

total trip 734 seconds
Trip in GeoFencing area 734 seconds


Figure(layout=FigureLayout(height='420px'))

# Geo Fencing Example 3

### : *Available to fence even whole street, i.e., 7th street*

In [17]:
#map_geofence(geo_key['sevent'], seventh_st)

# Geo Fencing Example 4

### : *A Geo Fenced Gas Station where the Bus is filled the fuel at*

In [18]:
#map_geofence(geo_key['statio'], gas_station)